# INTRODUCTION
In this notebook we will be using the following packages:
  * [pyspark](https://spark.apache.org/docs/latest/api/python/pyspark.html)
  * [pandas](https://pandas.pydata.org/pandas-docs/stable/index.html)
  * [pyarrow](https://arrow.apache.org/docs/python/index.html)

We will be using the following data:
  * dressipi_recsys2022

The dataset is split in 4 csv files:
   * candidate_items : contains the candidate items for each user
   * item_features : contains the features for each item (items can have multiple features)
   * train_purchases : contains the purchases for each user
   * train_sessions : contains the sessions for each user



In [1]:
# Uncomment to install the required packages
# %pip install pyspark
# %pip install pyarrow


## Loading the dataset

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import numpy as np
import os

# start spark session
os.environ['PYSPARK_SUBMIT_ARGS'] ="--conf spark.driver.memory=2g  pyspark-shell"
# spark = SparkSession.builder.appName("Python Spark SQL basic example").config("spark.some.config.option", "some-value").getOrCreate()
spark = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("AppName") \
    .config('spark.ui.port', '4050')\
    .getOrCreate()

candidate_items, item_features, train_purchases, train_sessions = [None] * 4
datasets = [candidate_items, item_features, train_purchases, train_sessions]

#load all datasets
def load_candidate_items():
    global candidate_items
    candidate_items = spark.read.csv("dressipi_recsys2022/candidate_items.csv", header=True)
    candidate_items = candidate_items.withColumn("item_id", candidate_items["item_id"].cast("int"))

def load_item_features():
    global item_features
    item_features = spark.read.csv("dressipi_recsys2022/item_features.csv", header=True)
    item_features = item_features.withColumn("item_id", item_features["item_id"].cast("int"))
    item_features = item_features.withColumn("feature_category_id", item_features["feature_category_id"].cast("int"))
    item_features = item_features.withColumn("feature_value_id", item_features["feature_value_id"].cast("int"))

def load_train_purchases():
    global train_purchases
    train_purchases = spark.read.csv("dressipi_recsys2022/train_purchases.csv", header=True)
    train_purchases = train_purchases.withColumn("session_id", train_purchases["session_id"].cast("int"))
    train_purchases = train_purchases.withColumn("item_id", train_purchases["item_id"].cast("int"))
    train_purchases = train_purchases.withColumn("date", train_purchases["date"].cast("timestamp"))

def load_train_sessions():
    global train_sessions
    train_sessions = spark.read.csv("dressipi_recsys2022/train_sessions.csv", header=True)
    train_sessions = train_sessions.withColumn("session_id", train_sessions["session_id"].cast("int"))
    train_sessions = train_sessions.withColumn("item_id", train_sessions["item_id"].cast("int"))
    train_sessions = train_sessions.withColumn("date", train_sessions["date"].cast("timestamp"))

def load_datasets():
    global datasets
    load_candidate_items()
    load_item_features()
    load_train_purchases()
    load_train_sessions()
    datasets = [candidate_items, item_features, train_purchases, train_sessions]

load_datasets()


## Quick look at the data

### train_sessions

In [3]:
train_sessions.printSchema()
train_sessions.show(5)

root
 |-- session_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- date: timestamp (nullable = true)

+----------+-------+--------------------+
|session_id|item_id|                date|
+----------+-------+--------------------+
|         3|   9655|2020-12-18 21:25:...|
|         3|   9655|2020-12-18 21:19:...|
|        13|  15654|2020-03-13 19:35:...|
|        18|  18316|2020-08-26 19:18:...|
|        18|   2507|2020-08-26 19:16:...|
+----------+-------+--------------------+
only showing top 5 rows



### train_purchases

In [4]:
train_purchases.show(5)

+----------+-------+--------------------+
|session_id|item_id|                date|
+----------+-------+--------------------+
|         3|  15085|2020-12-18 21:26:...|
|        13|  18626|2020-03-13 19:36:...|
|        18|  24911|2020-08-26 19:20:...|
|        19|  12534|2020-11-02 17:16:...|
|        24|  13226|2020-02-26 18:27:...|
+----------+-------+--------------------+
only showing top 5 rows



### item_features

In [5]:
item_features.show(5)

+-------+-------------------+----------------+
|item_id|feature_category_id|feature_value_id|
+-------+-------------------+----------------+
|      2|                 56|             365|
|      2|                 62|             801|
|      2|                 68|             351|
|      2|                 33|             802|
|      2|                 72|              75|
+-------+-------------------+----------------+
only showing top 5 rows



### candidate_items

In [6]:
candidate_items.show(5)

+-------+
|item_id|
+-------+
|      4|
|      8|
|      9|
|     19|
|     20|
+-------+
only showing top 5 rows



# Part 1 : Pipeline

## Checking for null values

In [7]:
# Are there missing values (na or null) in any of those dataframes ? print the number of missing values
print("candidate_items:", candidate_items.filter(candidate_items["item_id"].isNull()).count())
print("item_features:", item_features.filter(item_features["item_id"].isNull()).count())
print("train_purchases:", train_purchases.filter(train_purchases["item_id"].isNull()).count())
print("train_sessions:", train_sessions.filter(train_sessions["item_id"].isNull()).count())

candidate_items: 0
item_features: 0
train_purchases: 0
train_sessions: 0


## Feature engineering

1) Month

In [8]:
month = train_sessions.rdd.map(lambda x: (x["session_id"], int(x["date"].strftime("%m")))).reduceByKey(min)
print(month.take(5))

[(18, 8), (24, 2), (28, 5), (36, 6), (42, 3)]


2) Day of month

In [9]:
day_of_month = train_sessions.rdd.map(lambda x: (x["session_id"], int(x["date"].strftime("%d")))).reduceByKey(min)
print(day_of_month.take(5))

[(18, 26), (24, 26), (28, 18), (36, 21), (42, 1)]


3) Weekday

In [10]:
weekday = train_sessions.rdd.map(lambda x: (x["session_id"], int(x["date"].strftime("%w")))).reduceByKey(min)
print(weekday.take(5))

[(18, 3), (24, 3), (28, 1), (36, 0), (42, 1)]


4) Hour period

In [11]:
hour_period = train_sessions.rdd.map(lambda x: (x["session_id"], int(x["date"].strftime("%H")))).reduceByKey(min)
print(hour_period.take(5))

[(18, 19), (24, 17), (28, 12), (36, 10), (42, 15)]


5) Season (Meteorological)

In [12]:
def get_season(month):
    if month == 12 or month <= 2: return 0
    elif 2 < month <= 5: return 1
    elif 5 < month <= 8: return 2
    elif 8 < month <= 11: return 3

season = train_sessions.rdd.map(lambda x: (x["session_id"], get_season(int(x["date"].strftime("%m"))))).reduceByKey(min)
print(season.take(5))

[(18, 2), (24, 0), (28, 1), (36, 2), (42, 1)]


6) Average time between consecutive item views

In [13]:
def get_average_time(dates):
    import datetime
    dates = sorted(list(dates))
    avgs = [dates[i+1] - dates[i] for i in range(len(dates)-1)]
    return (sum(avgs, datetime.timedelta())/len(avgs)).total_seconds() if len(avgs) > 0 else 0

average_time = train_sessions.rdd.map(lambda x: (x["session_id"], x["date"])).groupByKey().mapValues(get_average_time)

average_time.take(5)

[(18, 81.8005), (24, 462.983375), (28, 29.171), (36, 43.074), (42, 40.116)]

7) Number of distinct items

In [14]:
distinct_nb = train_sessions.rdd.map(lambda x: (x["session_id"], x["item_id"])).groupByKey().mapValues(lambda x: len(set(x)))
distinct_nb.take(5)

[(18, 3), (24, 8), (28, 3), (36, 2), (42, 4)]

8) Number of repetitive items

In [15]:
repetitive_nb = train_sessions.rdd.map(lambda x: (x["session_id"], x["item_id"])).groupByKey().mapValues(lambda x: len(list(x)) - len(set(x)))
repetitive_nb.take(5)

[(18, 0), (24, 1), (28, 1), (36, 0), (42, 0)]

9) Same category

In [16]:
item_features_rdd = item_features.rdd.map(lambda x: (x["item_id"], (x["feature_category_id"], x["feature_value_id"]))).groupByKey().mapValues(lambda x: [(a,b) for a, b in x])
item_features_rdd.take(1)

[(2,
  [(56, 365),
   (62, 801),
   (68, 351),
   (33, 802),
   (72, 75),
   (29, 123),
   (16, 38),
   (50, 76),
   (61, 462),
   (53, 6),
   (7, 394),
   (69, 885),
   (47, 123)])]

In [17]:
def get_same_category(x):
    dico = dict()
    for item in x:
        for cat in item:
            if cat in dico:
                dico[cat] += 1
            else:
                dico[cat] = 0

    res = 0
    for val in dico.values():
        if val > 0:
            res += 1

    return res

same_category = item_features_rdd.join(train_sessions.rdd.map(lambda x: (x["item_id"], x["session_id"]))).map(lambda x: (x[1][1], x[1][0])).groupByKey().mapValues(get_same_category)
same_category.take(5)

[(7588, 34), (9396, 54), (28268, 66), (59544, 13), (61536, 15)]

10) Different category

In [18]:
def get_different_category(x):
    dico = dict()
    for item in x:
        for cat in item:
            if cat in dico:
                dico[cat] += 1
            else:
                dico[cat] = 0

    res = 0
    for val in dico.values():
        if val == 0:
            res += 1

    return res

diff_category = item_features_rdd.join(train_sessions.rdd.map(lambda x: (x["item_id"], x["session_id"]))).map(lambda x: (x[1][1], x[1][0])).groupByKey().mapValues(get_different_category)
diff_category.take(5)

[(3481568, 21), (4252364, 12), (1531580, 74), (1200416, 18), (3642496, 28)]

11) Last item

In [19]:
def get_last_item(items):
    return max(list(items), key=lambda i: i[1])[0]

last_item = train_sessions.rdd.map(lambda x: (x["session_id"], (x["item_id"], x["date"]))).groupByKey().mapValues(lambda x: max(x, key=lambda i: i[1])[0])
last_item.take(5)

[(842, 22223), (846, 1787), (1962, 23592), (2654, 25931), (2942, 10537)]

12) Most present category

In [20]:
def get_most_present_category(categories_i):
    categories = [cat for cat_i in categories_i for cat in cat_i]    
    categories.sort()
    
    most_viewed = (None, -1)
    last_viewed = categories[0]
    cnt = 0
    
    for category in categories:
        if last_viewed != category:
            if cnt > most_viewed[1]:
                most_viewed = (last_viewed, cnt)
                cnt = 1
                last_viewed = category
        else:
            cnt += 1
            
    if cnt > most_viewed[1]:
        most_viewed = (last_viewed, cnt)
    
    return most_viewed[0]
    
most_present_category = item_features_rdd.join(train_sessions.rdd.map(lambda x: (x["item_id"], x["session_id"]))).map(lambda x: (x[1][1], x[1][0])).groupByKey().mapValues(get_most_present_category)
most_present_category.take(5)

[(1200432, (3, 793)),
 (3161856, (3, 793)),
 (433624, (3, 793)),
 (1074668, (3, 793)),
 (2063408, (1, 461))]

13) Most viewed item

In [21]:
def get_most_viewed_item(items):
    items = list(items)
    items.sort()
    most_viewed = (None, -1)
    
    last_viewed = items[0]
    cnt = 0
    
    for item in items:
        if last_viewed != item:
            if cnt > most_viewed[1]:
                most_viewed = (last_viewed, cnt)
                cnt = 1
                last_viewed = item
        else:
            cnt += 1
            
    if cnt > most_viewed[1]:
        most_viewed = (last_viewed, cnt)
    
    return most_viewed[0]
    
most_viewed_item = train_sessions.rdd.map(lambda x: (x["session_id"], x["item_id"])).groupByKey().mapValues(get_most_viewed_item)
most_viewed_item.take(5)

[(18, 2507), (24, 2927), (28, 11529), (36, 25417), (42, 10395)]

14) Length of the session

In [22]:
length = train_sessions.rdd.map(lambda x: (x["session_id"], x["item_id"])).groupByKey().mapValues(len)
length.take(5)

[(18, 3), (24, 9), (28, 4), (36, 2), (42, 4)]

15) Duration of the session

In [23]:
def get_session_duration(dates):
    dates = list(dates)
    dates.sort()
    return (dates[-1] - dates[0]).total_seconds() if len(dates) >= 2 else 0

duration = train_sessions.rdd.map(lambda x: (x["session_id"], x["date"])).groupByKey().mapValues(get_session_duration)
duration.take(5)

[(18, 163.601), (24, 3703.867), (28, 87.513), (36, 43.074), (42, 120.348)]

16) a

17) a

18) a

19) a

20) a

BIG RDD

In [24]:
session_item_id = train_purchases.rdd.map(lambda x: (x["session_id"], x["item_id"]))
features = [month, day_of_month, weekday, hour_period, season, average_time, distinct_nb, repetitive_nb, same_category, diff_category, last_item, most_present_category, most_viewed_item, length, duration]
features_name = ["month", "day_of_month", "weekday", "hour_period", "season", "average_time", "distinct_nb", "repetitive_nb", "same_category", "diff_category", "last_item", "most_present_category", "most_viewed_item", "length", "duration"]

def get_BIG_RDD():
    temp = session_item_id
    for feature in features:
        temp = temp.join(feature).mapValues(lambda x: tuple(list(x[0])+[x[1]]) if isinstance(x[0], tuple) else x)
    return temp

BIG_RDD = get_BIG_RDD().cache()

In [25]:
BIG_RDD.take(1)

[(3255840,
  (25186,
   8,
   8,
   6,
   12,
   2,
   1282.361333,
   6,
   1,
   38,
   31,
   26382,
   (4, 618),
   5673,
   7,
   7694.168))]

# Part 2 : Feature selection


## Ranking algorithm

In [26]:
def pearson_reduce(a, b):
    n = a[0] + b[0]
    x = a[1] + b[1]
    y = a[2] + b[2]
    x2 = a[3] + b[3]
    y2 = a[4] + b[4]
    xy = a[5] + b[5]

    return n, x, y, x2, y2, xy

def calculate_pearson(a):
    import math

    n = a[0]
    x = a[1]
    y = a[2]
    x2 = a[3]
    y2 = a[4]
    xy = a[5]

    return (n*xy - x * y) / (math.sqrt((n*x2 - (x**2)) * (n*y2 - (y**2))))

def compute_pearson(rdd):
    temp = rdd.flatMap(lambda x: [(i, (x[1][i+1], x[1][0])) for i in range(len(x[1])-1)]).\
        mapValues(lambda a: (1, a[0], a[1], a[0]**2, a[1]**2, a[0]*a[1])).\
        reduceByKey(pearson_reduce).\
        mapValues(calculate_pearson).collect()
    return temp

In [27]:
features_score = compute_pearson(BIG_RDD)
features_score.sort(key=lambda x: abs(x[1]), reverse=True)

print("Features ranking:")
for feat in features_score:
    print(f"\t- {features_name[feat[0]]}: {feat[1]}")

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 123.0 failed 1 times, most recent failure: Lost task 0.0 in stage 123.0 (TID 390, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/usr/local/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/anaconda3/lib/python3.7/site-packages/pyspark/rdd.py", line 2499, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/anaconda3/lib/python3.7/site-packages/pyspark/rdd.py", line 2499, in pipeline_func
    return func(split, prev_fun

TEST Mutual Information

In [ ]:
df.select("item_id").toPandas()

In [1]:
from sklearn.feature_selection import mutual_info_classif as MIC

mi_score = MIC(df.select("month", "day_of_month", "weekday", "hour_period", "season", "distinct_nb", "repetitive_nb", "same_category", "diff_category", "last_item", "most_viewed_item", "length").toPandas(), df.select("item_id").toPandas())

print(mi_score)

NameError: name 'df' is not defined

TEST Covariance

In [55]:
from pyspark.sql.types import Row

def f(x):
    features_name = ["month", "day_of_month", "weekday", "hour_period", "season", "average_time", "distinct_nb", "repetitive_nb", "same_category", "diff_category", "last_item", "most_present_category", "most_viewed_item", "length", "duration"]
    dico = {}
    dico["session_id"] = x[0]
    dico["item_id"] = x[1][0]
    for i in range(len(features_name)):
        dico[features_name[i]] = x[1][i+1]
    return dico
        
        
df = BIG_RDD.map(lambda x: Row(**f(x))).toDF()
df.printSchema()
# df.show(5)

root
 |-- average_time: double (nullable = true)
 |-- day_of_month: long (nullable = true)
 |-- diff_category: long (nullable = true)
 |-- distinct_nb: long (nullable = true)
 |-- duration: double (nullable = true)
 |-- hour_period: long (nullable = true)
 |-- item_id: long (nullable = true)
 |-- last_item: long (nullable = true)
 |-- length: long (nullable = true)
 |-- month: long (nullable = true)
 |-- most_present_category: struct (nullable = true)
 |    |-- _1: long (nullable = true)
 |    |-- _2: long (nullable = true)
 |-- most_viewed_item: long (nullable = true)
 |-- repetitive_nb: long (nullable = true)
 |-- same_category: long (nullable = true)
 |-- season: long (nullable = true)
 |-- session_id: long (nullable = true)
 |-- weekday: long (nullable = true)



In [47]:
# sdi = df.select("stddev ('item_id')")
print(sdi)
for feature in features_name:
    print(f"{feature} : {df.cov('item_id', feature)}")

AnalysisException: "cannot resolve '`stddev ('item_id')`' given input columns: [length, season, average_time, last_item, most_viewed_item, repetitive_nb, most_present_category, session_id, month, day_of_month, same_category, diff_category, distinct_nb, duration, weekday, item_id, hour_period];;\n'Project ['stddev ('item_id')]\n+- LogicalRDD [average_time#387, day_of_month#388L, diff_category#389L, distinct_nb#390L, duration#391, hour_period#392L, item_id#393L, last_item#394L, length#395L, month#396L, most_present_category#397, most_viewed_item#398L, repetitive_nb#399L, same_category#400L, season#401L, session_id#402L, weekday#403L], false\n"

## Forward feature selection

# Part 3 : Model

In [62]:
#from pyspark.ml.classification import NaiveBayes
from pyspark.mllib.classification import NaiveBayes, NaiveBayesModel
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree

In [77]:
test = BIG_RDD.map(lambda x: LabeledPoint(x[1][0], list(x[1][1:-4])+[str(x[1][4])]+list(x[1][-3:])))

In [82]:

model = NaiveBayes.train(test)
#nb = NaiveBayes()
#nb.setFeaturesCol("month")
#model = nb.fit(df)


In [83]:
# https://spark.apache.org/docs/latest/mllib-naive-bayes.html

predictionAndLabel = BIG_RDD.map(lambda x: (model.predict(list(x[1][1:-4])+[str(x[1][4])]+list(x[1][-3:])), x[1][0]))
accuracy = 1.0 * predictionAndLabel.filter(lambda pl: pl[0] == pl[1]).count() / test.count()
print('model accuracy {}'.format(accuracy))

model accuracy 0.001247
